In [ ]:
import argparse
import torch
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import BertConfig, BertForSequenceClassification, AdamW, get_scheduler
# from datasets import load_metric
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.datasets import load_svmlight_file

In [ ]:
df = pd.read_csv("annotated_corpus.csv")

In [ ]:
#creating the hallucination 

hall = []
for j in df.values:
    if j[4] == 0 and j[5] == 0 and j[6] == 0 and j[7] == 0 and j[8] == 0:
        hall.append(0)
    else:
        hall.append(1)

df['hallucinate'] = hall

In [ ]:
# Undersampling 
df1 = df.loc[df.hallucinate == 0]
df1_ = df1.sample(532)
df2_ = df.loc[df.hallucinate == 1]

df = pd.concat([df1_, df2_], ignore_index = True)

In [ ]:
# Oversampling 

max_size = df['hallucinate'].value_counts().max()
max_size

lst = [df]
for class_index, group in df.groupby('hallucinate'):
    lst.append(group.sample(max_size-len(group), replace=True))
    
frame_new = pd.concat(lst)

In [ ]:
frame_new.hallucinate.value_counts()

df = frame_new.copy()

In [ ]:
# normal concatenation of SRC and TGT strings 
df['src_mt'] = df['src'].astype(str) + df['mt'].astype(str)

In [ ]:
# concatenating source and translated strings using SRC and TGT tags
df['src_mt'] = "SRC: " + df['src'].astype(str) + " TGT: " + df['mt'].astype(str)

In [ ]:
# PoS tagging 
tag_english = []
tag_german = []

src_pos = []
mt_pos = []



import spacy

nlp1 = spacy.load("en_core_web_lg")
nlp2 = spacy.load("de_core_news_lg")

for i in df.mt.values:
    doc = nlp1(i)
    p = []
    for token in doc:
        p.append(token.pos_)
    tag_english.append(' '.join(x.strip() for x in p))
    
for i in df.src.values:
    doc = nlp2(i)
    p = []
    for token in doc:
        p.append(token.pos_)
    tag_german.append(' '.join(x.strip() for x in p))

    
for i, j in zip(df.src.values, tag_german):
    p = i.split()
    q = j.split()
    ss = []
    for k,l in zip(p,q):
        ss.append(str(k) + "," + str(l))
    src_pos.append(" ".join(ss))

    
for i, j in zip(df.mt.values, tag_english):
    p = i.split()
    q = j.split()
    ss = []
    for k,l in zip(p,q):
        ss.append(str(k) + "," + str(l))
    mt_pos.append(" ".join(ss))

In [ ]:
df['src_pos'] = src_pos
df['mt_pos'] = mt_pos
df['src_mt_pos'] = df['src_pos'].astype(str) + df['mt_pos'].astype(str)

In [ ]:
# NER tagging  
tag_english = []
tag_german = []


import spacy

nlp1 = spacy.load("en_core_web_lg")
nlp2 = spacy.load("de_core_news_lg")


for i in df.mt.values:
    doc = nlp1(i)
    p = []
    q = []
    x = i
    for ents in doc.ents:
#         print(ents)
        if (ents.label_ == 'LOC') or (ents.label_ == 'PER') or (ents.label_ == 'ORG') or (ents.label_ == 'MISC'):
#             print(ents.text)
#             print(ents.text
            x = x.replace(ents.text, ents.label_ + "_" + ents.text)
    tag_english.append(x)
    
for i in df.src.values:
    doc = nlp2(i)
    p = []
    q = []
    x = i
    for ents in doc.ents:
#         print(ents)
        if (ents.label_ == 'LOC') or (ents.label_ == 'PER') or (ents.label_ == 'ORG') or (ents.label_ == 'MISC'):
#             print(ents.text)
            x = x.replace(ents.text, ents.label_ + "_" + ents.text)
    tag_german.append(x)

In [ ]:
df['src_ner'] = tag_german
df['mt_ner'] = tag_english

df['src_mt_ner'] = df['src_ner1'].astype(str) + df['mt_ner1'].astype(str)

In [ ]:
#train test split

In [ ]:
import numpy as np 
train, validate, test = np.split(df.sample(frac=1), [int(.8*len(df)), int(0.9*len(df))])

In [ ]:
# prompt engineering approach 

In [ ]:
train['src_mt'] = "German source: " + train['src'].astype(str) + " English translation: " + train['mt'].astype(str) + " Hallucination: " + train['hallucinate'].astype(str)

validate['src_mt'] = "German source: " + validate['src'].astype(str) + " English translation: " + validate['mt'].astype(str) + " Hallucination: ?"

test['src_mt'] = "German source: " + test['src'].astype(str) + " English translation: " + test['mt'].astype(str) + " Hallucination: ?"


In [ ]:
train['src_mt'] = "SRC: " + train['src'].astype(str) + " MT: " + train['mt'].astype(str)
validate['src_mt'] = "SRC: " + validate['src'].astype(str) + " MT: " + validate['mt'].astype(str)
test['src_mt'] = "SRC: " + test['src'].astype(str) + " MT: " + test['mt'].astype(str)

In [ ]:
# example here for only case, change values based on the type of data to be used 

sentences_train = train.src_mt.values
sentences_dev = validate.src_mt
sentences_test = test.src_mt.values

In [ ]:
labels_train = train.hallucinate.values
labels_dev = validate.hallucinate.values
labels_test = test.hallucinate.values

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_train = []
attention_masks_train = []

# For every sentence...
for sent in sentences_train:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_train.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.tensor(labels_train)

In [ ]:
input_ids_dev = []
attention_masks_dev = []

for sent in sentences_dev:
    encoded_dict = tokenizer.encode_plus(
                        sent,#.tolist()[0],                     
                        add_special_tokens = True, 
                        max_length = 512,       
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                        truncation=True
                   )
    
    input_ids_dev.append(encoded_dict['input_ids'])
    
    attention_masks_dev.append(encoded_dict['attention_mask'])

input_ids_dev = torch.cat(input_ids_dev, dim=0)
attention_masks_dev = torch.cat(attention_masks_dev, dim=0)
labels_dev = torch.tensor(labels_dev)

In [ ]:
input_ids_test = []
attention_masks_test = []

for sent in sentences_test:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True,
                        max_length = 512,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                        truncation=True
                   )
    
    input_ids_test.append(encoded_dict['input_ids'])
    
    attention_masks_test.append(encoded_dict['attention_mask'])

input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
labels_test = torch.tensor(labels_test)

### Data 

In [ ]:
from torch.utils.data import TensorDataset, random_split

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16


train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size,
            
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
)

test_dataloader = DataLoader(
            test_dataset,
            sampler = SequentialSampler(test_dataset), 
            batch_size = batch_size 
)

In [ ]:
import torch.nn as nn

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-uncased", 
    num_labels = 2, 
    output_attentions = False, 
    output_hidden_states = True, 
)

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
type(model)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model= nn.DataParallel(model)
model.to(device)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 4e-5, 
                  eps = 1e-8, 
                  weight_decay = 0.003
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

### Training loop

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

total_t0 = time.time()
 

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()
    output_model = "bert_train_{}.pth".format(epoch_i)

    for step, batch in enumerate(train_dataloader):

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        loss = outputs[0]
        logits = outputs[1]
        
        loss = torch.mean(loss)
        total_train_loss += loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)
 
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
        
        loss = torch.mean(loss)
        total_eval_loss += loss

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    ) 

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import pandas as pd

pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)

df_stats = df_stats.set_index('epoch')

df_stats

In [ ]:
import matplotlib.pyplot as plt
# matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

### Prediction on test set 

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids_test)))

model.eval()

predictions , true_labels = [], []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
  
    b_input_ids, b_input_mask, b_labels = batch
  
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
  
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

In [ ]:
from sklearn.metrics import accuracy_score

flat_predictions = np.concatenate(predictions, axis=0)

flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)

acc = accuracy_score(flat_true_labels, flat_predictions)

print(acc)

In [ ]:
flat_predictions 

In [ ]:
flat_true_labels

In [ ]:
test['pred'] = flat_predictions 
test['true'] = flat_true_labels

In [ ]:
test.to_csv("BERT_pred_pos.csv")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_true_labels,flat_predictions))